Problem Statement.

You are given a rows x cols matrix grid. Initially, you are located at the top-left corner (0, 0), and in each step, you can only move right or down in the matrix.

Among all possible paths starting from the top-left corner (0, 0) and ending in the bottom-right corner (rows - 1, cols - 1), find the path with the maximum non-negative product. The product of a path is the product of all integers in the grid cells visited along the path.

Return the maximum non-negative product modulo 109 + 7. If the maximum product is negative return -1.

Notice that the modulo is performed after getting the maximum product.

 

Example 1:

Input: grid = [[-1,-2,-3],
               [-2,-3,-3],
               [-3,-3,-2]]
Output: -1
Explanation: It's not possible to get non-negative product in the path from (0, 0) to (2, 2), so return -1.

Example 2:

Input: grid = [[1,-2,1],
               [1,-2,1],
               [3,-4,1]]
Output: 8
Explanation: Maximum non-negative product is in bold (1 * 1 * -2 * -4 * 1 = 8).

Example 3:

Input: grid = [[1, 3],
               [0,-4]]
Output: 0
Explanation: Maximum non-negative product is in bold (1 * 0 * -4 = 0).

Example 4:

Input: grid = [[ 1, 4,4,0],
               [-2, 0,0,1],
               [ 1,-1,1,1]]
Output: 2
Explanation: Maximum non-negative product is in bold (1 * -2 * 1 * -1 * 1 * 1 = 2).

 

Constraints:

    1 <= rows, cols <= 15
    -4 <= grid[i][j] <= 4

# BFS with memo - O(2 ^ N) runtime, O(2 ^ N) space where N is the number of cells

In [1]:
from typing import List
from collections import defaultdict, deque

class Solution:
    def maxProductPath(self, grid: List[List[int]]) -> int:
        m, n = len(grid), len(grid[0])
        
        queue = deque([(0, 0, grid[0][0])])
        res = float('-inf')
        visited = defaultdict(set)
        
        while queue:
            r, c, prod = queue.popleft()
            if (r, c) == (m-1, n-1):
                res = max(res, prod)
                
            for r1, c1 in [(r+1, c), (r, c+1)]:
                if 0 <= r1 < m and 0 <= c1 < n:
                    val = prod*grid[r1][c1]
                    if (r1, c1) not in visited or val not in visited[(r1, c1)]:
                        queue.append((r1, c1, val))
                        visited[(r1, c1)].add(val)
                    
        return res % (10**9 + 7) if res >= 0 else -1

# Top Down DP - O(M * N) runtime, O(M * N) space

In [2]:
from typing import List
from functools import lru_cache

class Solution:
    def maxProductPath(self, grid: List[List[int]]) -> int:
        m, n = len(grid), len(grid[0])
        
        @lru_cache(maxsize=None)
        def fn(i, j): 
            """Return maximum & minimum products ending at (i, j)."""
            if i == 0 and j == 0: return grid[0][0], grid[0][0]
            if i < 0 or j < 0: return float('-inf'), float('inf')
            if grid[i][j] == 0: return 0, 0
            mx1, mn1 = fn(i-1, j) # from top
            mx2, mn2 = fn(i, j-1) # from left 
            mx, mn = max(mx1, mx2)*grid[i][j], min(mn1, mn2)*grid[i][j]
            return (mx, mn) if grid[i][j] > 0 else (mn, mx)
        
        mx, _ = fn(m-1, n-1)
        return -1 if mx < 0 else mx % 1_000_000_007

# Bottom Up DP - O(M * N) runtime, O(M * N) space

In [3]:
from typing import List

class Solution:
    def maxProductPath(self, grid: List[List[int]]) -> int:
        m, n = len(grid), len(grid[0])
        dp = [[0, 0] * n for _ in range(m)]
        dp[0][0] = [grid[0][0], grid[0][0]]

        for j in range(1, n):
            dp[0][j] = [dp[0][j - 1][0] * grid[0][j], dp[0][j - 1][1] * grid[0][j]]

        for i in range(1, m):
            dp[i][0] = [dp[i - 1][0][0] * grid[i][0], dp[i - 1][0][1] * grid[i][0]]

        for i in range(1, m):
            for j in range(1, n):
                maxVal = max(dp[i - 1][j][0], dp[i][j - 1][0]) * grid[i][j]
                minVal = min(dp[i - 1][j][1], dp[i][j - 1][1]) * grid[i][j]
                if grid[i][j] > 0:
                    dp[i][j] = [maxVal, minVal]
                else:
                    dp[i][j] = [minVal, maxVal]
                    
        return dp[m-1][n-1][0] % (10**9 + 7) if dp[m-1][n-1][0] >= 0 else -1

In [4]:
instance = Solution()
instance.maxProductPath([[1,-2,1], [1,-2,1], [3,-4,1]])

8